# Baseline de algoritmos de xtracción de palabras clave

## Algoritmos básicos: TextRank y Fractalidad

### Importes para librerías

In [1]:
from math import *
from math import sqrt
import string
# import csv
import operator
# import random
import pandas as pd
#librerias necesarias para text rank
from collections import OrderedDict
import numpy as np
import spacy
#import nlp

#Listado de STOPWORDS dependiendo del lenguaje
#from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.es.stop_words import STOP_WORDS

In [2]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 3.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
nlp = spacy.load('es_core_news_sm')

## Definición de la clase TextRank

In [4]:
class TextRank4Keyword():
    """Extract keywords from text"""

    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 100 # iteration steps
        self.node_weight = None # save keywords and its weight


    def set_stopwords(self, stopwords):
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True

    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences

    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab

    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs

    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())

    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1

        # Get Symmeric matrix
        g = self.symmetrize(g)

        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm

        return g_norm


    def get_keywords(self, number=10):
        """Print top number keywords"""
        keysw={}
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            keysw[key] =value
            if i > number:
                break
        return keysw


    def analyze(self, text,
                candidate_pos=['NOUN', 'PROPN'],
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""

        # Set stop words
        self.set_stopwords(stopwords)

        # Pare text by spaCy
        doc = nlp(text)

        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words

        # Build vocabulary
        vocab = self.get_vocab(sentences)

        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)

        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)

        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))

        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]

        self.node_weight = node_weight


## Definición de la función Grado de Fractalidad

In [5]:
#solamente se calcula el grado de fractalidad de las palabras que tengan mas de uno de frecuencia
def fractalidad(palabras,vocabulario,frec,dist):
    N=len(palabras)                                     #El número de tokens de todo el texto
    gf={}
    cajas_index=set()
    voc=[]                                             #la variable voc contendra cada sintagma con frecuencia mayor que 1, por que las otras palabras tendrán 0 de grado de fractaldiad
    for p in vocabulario:                              #Esto se puede hacer fuera del algoritmo, pero se incluye para evitar ese calculo innecesario
        if(p not in voc):
            if(frec[p]>1):
                if(p not in STOP_WORDS):
                    if(len(p)>1):
                        voc.append(p)
    # print("Text size: ",N)
    # print("Vocabulary: ",len(voc))
    for p in voc:
        rcajas=dist[p]
        M=frec[p]
        dfw=0.0
        nsh=0.0
        for s in range(1,N+1):
            noc=0
            for e in rcajas:
                cajas_index.add(ceil(int(e)/s))
            noc=len(cajas_index)
            cajas_index.clear()
            ns=N/s
            if(M<=ns):
                nsh=M
            else:
                nsh=M/(1+(M-1)/(N-1)*(s-1))
            dfw=dfw+fabs(log(nsh/noc))
        gf[p]=dfw
    return gf    #regresamos un diccionario

In [6]:
def distribucion(palabras,vocabulario):
    N=len(palabras)
    ncajas=[]
    cajas={}
    frecuencias={}
    for p in vocabulario:
        ncajas.clear()
        i=0
        M=palabras.count(p)
        while(i<N):
            if(p == palabras[i]):
                ncajas.append(i+1)
            i=i+1
        frecuencias[p]=M
        cajas[p]=ncajas[:]
    return frecuencias,cajas

## Lectura de archivo de entrada

In [7]:
#Lectura de archivo para generación de vocabulario
def cargar_datos(filename):
    with open(filename, encoding="utf-8-sig") as f:
        texto = pd.read_csv(f)
        texto = texto['Texto']
        for row in texto:
            #Pasar a minusculas
            row = ' '.join(row)
            row = row.lower()
    #Eliminar puntuación
    texto =' '.join(texto)
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    texto = texto.translate(str.maketrans('', '', '¿¡—“”0123456789’'))
    palabras = texto.split()
    textop=""
    #rearmamos el texto debido a que existen carácteres especiales
    for w in palabras:
        textop=textop+w+' '
    return textop

### Lectura de documentos

In [8]:
def lee_documento(filename='NULL',texto=''):
    if filename != 'NULL':
        texto=cargar_datos(filename)
    #obtenemos el vocabulario
    tokens=texto.split()
    vocabulario=[]
    for t in tokens:
        if(t not in vocabulario):
            vocabulario.append(t)
    #variables de procesamiento
    dist={}
    frec={}
    frec,dist=distribucion(tokens,vocabulario)
    return frec,dist,tokens,vocabulario,texto

## Ejecución de algoritmos

In [9]:
#Lectura de documento para el corpus de suicidio
frec,dist,tokens,vocabulario,texto = lee_documento('/home/m/MGP/maestria/proyecto_tesis/baseline/Datasets/SMS_DATA_ORIGINAL.csv')

In [10]:
print('Frecuencias:', len(frec))
print("Distribuciones: ", len(dist))
print("Tokens: ", len(tokens))
print('Vocabulario: ', len(vocabulario))
print('Texto: ', len(texto))


Frecuencias: 12564
Distribuciones:  12564
Tokens:  246686
Vocabulario:  12564
Texto:  1302315


### Grado de Fractalidad

In [11]:
#ejecución de algoritmo Grado de Fractalidad
def grado_de_fractalidad(tokens,vocabulario,frec,dist,regresa_kw=False,regresa_df=True,top_n=np.inf,escribe_arch=False):
    frac_x=fractalidad(tokens,vocabulario,frec,dist)
    sorted_x = sorted(frac_x.items(), key=operator.itemgetter(1), reverse=True)
    # print('Time GF: '+str(elapsed_time))

    #Imprimir y guardar resultados de GF
    if regresa_df:
        if top_n != np.inf:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x[:top_n]]
        else:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x]
        #Ordenar resultados por medida combinada
        df.sort(key=lambda x: x[3],reverse=True)
        if regresa_kw==False:
            df = [dato[0] for dato in df]
            by_MC=pd.DataFrame(df, columns=['word'])
        else:
            by_MC=pd.DataFrame(df, columns=['word','frecuency','Degree_of_fractality','Combined_measure'])
        if escribe_arch:
            by_MC.to_csv('GF.csv')
    else:
        if top_n != np.inf:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x[:top_n]]
        else:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x]
        #Ordenar resultados por medida combinada
        df.sort(key=lambda x: x[3],reverse=True)
        if regresa_kw==False:
            by_MC = [dato[0] for dato in df]
        else:
            by_MC = df
        if escribe_arch:
            print('\nNo se tiene implementada la escritura de archivo cuando regresa_df==False\n')
    return by_MC

def use_gf(texto,regresa_kw=False,regresa_df=False,top_n=np.inf,escribe_arch=False):
    tokens=texto.split()
    vocabulario=[]
    for t in tokens:
        if(t not in vocabulario):
            vocabulario.append(t)
    #variables de procesamiento
    dist={}
    frec={}
    frec,dist=distribucion(tokens,vocabulario)
    df = grado_de_fractalidad(tokens,vocabulario,frec,dist,regresa_kw,regresa_df,top_n,escribe_arch)
    return df

df1 = use_gf(texto,regresa_kw=True,regresa_df=True,top_n=338)
df1

,word,frecuency,Degree_of_fractality,Combined_measure
0,caca,23,211335.717556,287781.729340
1,Ansium,29,186659.515902,272970.502344
2,tendido,23,181585.856566,247270.515513
3,recogido,182,109372.307097,247189.221909
4,Naltrexona,16,197871.809551,238261.399885
...,...,...,...,...
333,racional,2,102125.064993,30742.707872
334,Gonzalo,2,102123.678699,30742.290556
335,fijación,2,102123.456101,30742.223547
336,inclusión,2,102120.066741,30741.203248


In [12]:
#Guarda los resultados del grado de fractalidad en un CSV. Siempre hay que cambiar la versión y comentar para no sobreescribir
# df1.to_csv('/home/m/MGP/maestria/proyecto_tesis/baseline/output/fractalidad_resultados_prueba3.csv')

### TextRank

In [13]:
#Cambio del máximo de tamaño de texto para que quepa el corpus (1.3 millones)
nlp.max_length = 1500000

In [14]:
#ejecución de algoritmo de TextRank
# start_time = time()
def use_TextRank(texto,regresa_kw=False,regresa_df=False,top_n=np.inf,escribe_arch=False):
    tr4w = TextRank4Keyword()
    tr4w.analyze(texto, candidate_pos = ['NOUN','PROPN'], window_size=4, lower=False)
    kwTR=tr4w.get_keywords(100)

    #Guardar resultados de TextRank
    if regresa_df:
        if top_n!=np.inf:
            if regresa_kw==True:
                salida = [[key, kwTR[key]] for key in kwTR.keys()][:top_n]
                dftr=pd.DataFrame(salida, columns=['word', 'Index'])
            else:
                salida = list(kwTR.keys())[:top_n]
                dftr=pd.DataFrame(salida, columns=['word'])
        else:
            if regresa_kw==True:
                salida = [[key, kwTR[key]] for key in kwTR.keys()]
                dftr=pd.DataFrame(salida, columns=['word', 'Index'])
            else:
                salida = list(kwTR.keys())
                dftr=pd.DataFrame(salida, columns=['word'])
    else:
        if top_n!=np.inf:
            if regresa_kw==True:
                dftr = [[key, kwTR[key]] for key in kwTR.keys()][:top_n]
            else:
                dftr = list(kwTR.keys())[:top_n]
        else:
            if regresa_kw==True:
                dftr = [[key, kwTR[key]] for key in kwTR.keys()]
            else:
                dftr = list(kwTR.keys())
        # elapsed_time = time() - start_time
        # print('Time TextRank: '+str(elapsed_time))
        if escribe_arch:
            dftr.to_csv('TextRank.csv')
    return dftr

dftr = use_TextRank(texto,top_n=338)
dftr

['cosas',
 'casa',
 'trabajo',
 'vida',
 'madre',
 'tiempo',
 'semana',
 'ansiedad',
 'dolor',
 'ganas',
 'cabeza',
 'noche',
 'problemas',
 'horas',
 'mañana',
 'medicación',
 'familia',
 'miedo',
 'gracias',
 'situación',
 'sueño',
 'años',
 'sensación',
 'gente',
 'momento',
 'persona',
 'ánimo',
 'problema',
 'cosa',
 'padre',
 'hora',
 'comida',
 'tratamiento',
 'marido',
 'pareja',
 'perro',
 'hija',
 'momentos',
 'médico',
 'mujer',
 'tensión',
 'cama',
 'doctora',
 'cuerpo',
 'pensamientos',
 'hijos',
 'dolores',
 'intento',
 'general',
 'mundo',
 'cambio',
 'consulta',
 'pastillas',
 'nervios',
 'hermana',
 'hijo',
 'mes',
 'enfermedad',
 'mierda',
 'tema',
 'noches',
 'personas',
 'hospital',
 'meses',
 'vacaciones',
 'cambios',
 'sigo',
 'pena',
 'cita',
 'año',
 'salud',
 'falta',
 'rato',
 'sábado',
 'dinero',
 'amigos',
 'amiga',
 'calle',
 'psiquiatra',
 'forma',
 'cuestionario',
 'daño',
 'fuerzas',
 'Madrid',
 'cansancio',
 'semanas',
 'mente',
 'tristeza',
 'coche',
 

In [15]:
#Guarda los resultados del TextRank en un txt. Siempre hay que cambiar la versión y comentar para no sobreescribir
# with open('/home/m/MGP/maestria/proyecto_tesis/baseline/output/TextRank_resultados_prueba3.txt','w') as output:
#     output.write(str(dftr))

## Comparación de los resultados de fractalidad y TextRank con el conjunto de Palabras Prototípicas

In [16]:
with open('/home/m/MGP/maestria/proyecto_tesis/baseline/Palabras_proto/clinical-prescores.csv', encoding="utf-8-sig") as f:
        palabrasProto = pd.read_csv(f)
        

In [17]:
palabrasProto

,Unnamed: 0,word,c-prescore
0,0,lormetazepam,-5.0
1,1,mg,-5.0
2,2,mirtazapina,-5.0
3,3,fluoxetina,-5.0
4,4,alprazolam,-5.0
...,...,...,...
334,334,aclaratorio,2.0
335,335,limpié,2.0
336,336,practicas,2.0
337,337,claros,2.0


### Función para crear la función de medida de Jaccard

In [18]:
#Medida Jaccard que mide la proporción de elementos de un conjunto en otro
def jaccard(s1: set, s2: set):
    return len(s1 & s2) / len(s1 | s2)   

In [20]:
jaccTodasFractalidad = jaccard(set(palabrasProto['word']), set(df1['word']))
jaccTodasTR = jaccard(set(palabrasProto['word']), set(dftr))


print('Jaccard de fractalidad: ', jaccTodasFractalidad)

print('Jaccard de TextRank: ', jaccTodasTR)


Jaccard de fractalidad:  0.01347305389221557
Jaccard de TextRank:  0.02320185614849188


In [21]:
def coinsidencia(conjuntoA, conjuntoB):
    matches = []
    for palabra in conjuntoA:
        if palabra in conjuntoB:
            matches.append(palabra)
    conteo = len(matches)
    return matches, conteo

In [22]:
coinFr, conteoFr = coinsidencia(list(palabrasProto['word']), list(df1['word']))
coinTR, conteoTR =coinsidencia(list(palabrasProto['word']), dftr)

In [23]:
print(coinFr)
print(conteoFr)
print(coinTR)
print(conteoTR)

['capsulas', 'acúfenos', 'calumnias', 'desconfío', 'dolorido', 'quiste', 'interrupciones', 'manchado', 'tratamientos']
9
['miligramo', 'noche', 'vida', 'hermano', 'hijo', 'padre', 'hermana', 'hija', 'marido', 'madre']
10
